In [12]:
import cv2
import numpy as np
import pandas as pd
import os
import json
import requests
import re
import torch
import torch.nn as nn
import torch.optim as optim
from urllib.parse import urlencode
from io import BytesIO
from zipfile import ZipFile
from sklearn.cluster import DBSCAN, KMeans
from typing import List, Dict, Union
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from torch.utils.data import Dataset, DataLoader
from torchvision.models import resnet50, ResNet50_Weights
from tqdm import tqdm
from collections import defaultdict

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

Для начала достанем файлы с Яндекс.Диска, делаю это сразу в проекте для удобства.

In [2]:
def download_zip(public_key, dist_path):
    base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'

    final_url = base_url + urlencode(dict(public_key=public_key))
    response = requests.get(final_url)
    download_url = response.json()['href']
    response = requests.get(download_url)

    zipfile = ZipFile(BytesIO(response.content))
    zipfile.extractall(path=dist_path)

In [3]:
download_zip("https://disk.yandex.ru/d/Be8jLxVcQZ70lQ",
             "/home/jupyter/datasphere/project")
download_zip("https://disk.yandex.ru/d/1sqxBbA1hRumDQ",
             "/home/jupyter/datasphere/project")

Аналогично скачаем разметку.

In [5]:
download_zip("https://disk.yandex.ru/d/PEOGKbj5qJmGlQ",
             "/home/jupyter/datasphere/project")

Теперь строит главный вопрос - как мы будем находить повторяющиеся отрезки в видео?

В целом, идея следующая:

1. Сначала разобъём видео по кадрам с помощью OpenCV библиотеки для того, чтоб можно было работать уже не с самим видео, а покадрово.

2. Далее, нам нужно извлечь признаки из картинки. В целом, это можно делать разными методами, но наиболее эффективный по сути - используя предобученную модель (ну и достаточно затратный, возьмём ResNet50 для этого и библиотеку PyTorch).

3. Применим следующий трюк - для кадров, находящихся в интервалах с заставкой сделаем метку 1, а для кадров не из заставки - метку 0. Таким образом, сводим нашу задачу к задаче классификации - то есть понять, принадлжит ли кадр заставке или нет. Обучить можем легко на трейне.

4. Для того, чтоб не решать одной моделью для всех сериалов (а там, очевидно, разные заставки), возьмём разные модели для разных сериалов.

5. Потом просто проверим, что вероятность интервала кадров (последовательно идущих) больше порога - это и обозначим за наш предикт.

6. Ну и метрику посчитаем по покрытию нужного нам интервала.

In [13]:
def load_dataset(data_dir: str, labels_path: str) -> List[Dict[str, Union[str, dict]]]:
    """
    Загружает датасет с видео и метками из единого JSON файла.
    
    data_dir: Путь к директории с подпапками, содержащими видеофайлы
    labels_path: Путь к JSON файлу с метками
    """
    dataset = []
    
    try:
        with open(labels_path, 'r') as f:
            all_labels = json.load(f)
    except (FileNotFoundError, json.JSONDecodeError) as e:
        print(f"Ошибка загрузки JSON файла {labels_path}: {e}")
        all_labels = {}
    
    for series_folder in os.listdir(data_dir):
        series_path = os.path.join(data_dir, series_folder)
        
        if not os.path.isdir(series_path):
            continue
            
        for video_file in os.listdir(series_path):
            if not video_file.lower().endswith(('.mp4', '.avi', '.mov', '.mkv')):
                continue
                
            video_path = os.path.join(series_path, video_file)
            video_name = os.path.splitext(video_file)[0]
            
            annotations = None
            if video_name in all_labels:
                annotations = all_labels[video_name]
            elif f"{series_folder}/{video_name}" in all_labels:
                annotations = all_labels[f"{series_folder}/{video_name}"]
            
            dataset.append({
                'series': series_folder,
                'video': video_path,
                'video_name': video_name,
                'annotations': annotations
            })
    
    return dataset

Теперь загрузим датасет

In [14]:
train_dataset = load_dataset(
    data_dir="data_train_short",
    labels_path="labels_json/train_labels.json"
)

test_dataset = load_dataset(
    data_dir="data_test_short",
    labels_path="labels_json/test_labels.json"
)

if train_dataset:
    print(f"Загружено {len(train_dataset)} видео из обучающего набора")
    for item in train_dataset[:3]:
        for key, value in item.items():
            print(f"{key} : {value}")

Загружено 80 видео из обучающего набора
series : -220020068_456255414
video : data_train_short/-220020068_456255414/-220020068_456255414.mp4
video_name : -220020068_456255414
annotations : {'url': 'https://vkvideo.ru/video-220020068_456255414', 'name': 'Баскетс. 4 сезон, 7 серия', 'start': '0:01:42', 'end': '0:01:47'}
series : -220020068_456249693
video : data_train_short/-220020068_456249693/-220020068_456249693.mp4
video_name : -220020068_456249693
annotations : {'url': 'https://vkvideo.ru/video-220020068_456249693', 'name': 'Бывaeт и xyжe. 1 сезон, 14 серия', 'start': '0:00:06', 'end': '0:00:10'}
series : -220020068_456255339
video : data_train_short/-220020068_456255339/-220020068_456255339.mp4
video_name : -220020068_456255339
annotations : {'url': 'https://vkvideo.ru/video-220020068_456255339', 'name': 'Анатомия скандала. 1 сезон. 2 серия.', 'start': '0:00:10', 'end': '0:00:20'}


Теперь проведём агрегирование по названию сериалов.

In [15]:
def extract_series_name(annotation_name: str) -> str:
    """Извлекает чистое название сериала из строки"""
    if not annotation_name:
        return "Unknown"
    
    match = re.split(r'[.,]', annotation_name)[0].strip()
    clean_name = re.sub(r'[\d\s-]+$', '', match).strip()
    
    return clean_name if clean_name else "Unknown"

def aggregate_by_series(dataset: list) -> dict:
    """Группирует видео по названиям сериалов"""
    series_dict = defaultdict(list)
    
    for item in dataset:
        if item.get('annotations') and item['annotations'].get('name'):
            series_name = extract_series_name(item['annotations']['name'])
        else:
            series_name = "Unknown"
        
        video_info = {
            'video_path': item['video'],
            'video_name': item['video_name'],
            'url': item['annotations'].get('url') if item.get('annotations') else None,
            'start_time': item['annotations'].get('start') if item.get('annotations') else None,
            'end_time': item['annotations'].get('end') if item.get('annotations') else None,
            'original_name': item['annotations'].get('name') if item.get('annotations') else None
        }
        
        series_dict[series_name].append(video_info)
    
    return dict(series_dict)

Прочекаем:

In [16]:
series_data_train = aggregate_by_series(train_dataset)
series_data_test = aggregate_by_series(test_dataset)

for series_name, videos in series_data_train.items():
    print(f"\nСериал: {series_name}")
    print(f"Количество видео: {len(videos)}")
    for video in videos[:3]:
        print(f"  - {video['video_name']}: {video['start_time']}-{video['end_time']}")


Сериал: Баскетс
Количество видео: 19
  - -220020068_456255414: 0:01:42-0:01:47
  - -220020068_456255402: 0:00:17-0:00:22
  - -220020068_456255403: 0:00:16-0:00:22

Сериал: Бывaeт и xyжe
Количество видео: 9
  - -220020068_456249693: 0:00:06-0:00:10
  - -220020068_456249732: 0:00:06-0:00:10
  - -220020068_456249739: 0:00:06-0:00:10

Сериал: Анатомия скандала
Количество видео: 4
  - -220020068_456255339: 0:00:10-0:00:20
  - -220020068_456255340: 0:00:11-0:00:21
  - -220020068_456255341: 0:00:10-0:00:21

Сериал: Блеск
Количество видео: 10
  - -220020068_456241755: 0:03:08-0:03:12
  - -220020068_456241756: 0:01:06-0:01:10
  - -220020068_456241758: 0:01:58-0:01:02

Сериал: Амepикaнcкий вaндaл
Количество видео: 4
  - -220020068_456241671: 0:03:47-0:03:10
  - -220020068_456241672: 0:03:51-0:03:15
  - -220020068_456241673: 0:02:52-0:02:17

Сериал: Бриджертоны
Количество видео: 6
  - -220020068_456256016: 0:04:32-0:04:38
  - -220020068_456256013: 0:03:20-0:03:26
  - -220020068_456256012: 0:04:1

Обучим всю модель

In [26]:
class SeriesIntroDataset(Dataset): # удобное представление датасета
    def __init__(self, features, labels):
        self.features = torch.FloatTensor(features)
        self.labels = torch.FloatTensor(labels)
        
    def __len__(self):
        return len(self.features)
    
    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

class SeriesClassifier(nn.Module): # классификатор
    def __init__(self, input_size=2048):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_size, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 1),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        return self.fc(x)

class SeriesAwareIntroDetector:
    def __init__(self, frame_interval=0.5, device='cuda'):
        self.frame_interval = frame_interval
        self.models = {} # каждая модель будет предсказывать свой сериал
        self.device = device
        self.feature_extractor = resnet50(weights=ResNet50_Weights.IMAGENET1K_V2)
        self.feature_extractor = nn.Sequential(*list(self.feature_extractor.children())[:-1])
        self.feature_extractor.eval().to(device)
        
    def time_to_seconds(self, time_str):
        h, m, s = map(int, time_str.split(':'))
        return h * 3600 + m * 60 + s
    
    def extract_features(self, frame): 
        with torch.no_grad():
            frame_tensor = torch.from_numpy(frame).permute(2, 0, 1).unsqueeze(0).float().to(self.device) # достаём признаки из кадра
            features = self.feature_extractor(frame_tensor).flatten()
            return features.cpu().numpy()
    
    def process_video(self, video_path, time_range=None):
        cap = cv2.VideoCapture(video_path) # вытаскиваем кадры
        fps = cap.get(cv2.CAP_PROP_FPS)
        frame_interval_frames = int(fps * self.frame_interval)
        
        features = []
        times = []
        labels = []
        
        frame_count = 0
        while True:
            ret, frame = cap.read()
            if not ret:
                break
                
            current_time = frame_count / fps
            
            if frame_count % frame_interval_frames == 0:
                frame_resized = cv2.resize(frame, (224, 224))
                frame_resized = frame_resized[:, :, ::-1]  # BGR to RGB
                frame_resized = (frame_resized / 255.0 - [0.485, 0.456, 0.406]) / [0.229, 0.224, 0.225]  # нормализуем
                
                feature = self.extract_features(frame_resized)
                features.append(feature)
                times.append(current_time)
                
                if time_range:
                    start, end = map(self.time_to_seconds, time_range)
                    labels.append(1 if start <= current_time <= end else 0) 
                    # классификация по принципу: 1, если принадлежит отрезку, иначе 0
            frame_count += 1
        
        cap.release()
        return np.array(features), np.array(times), np.array(labels)
    
    def train_model(self, X_train, y_train, X_val, y_val, series_name, epochs=10):
        model = SeriesClassifier().to(self.device) # будет находить отрезки
        criterion = nn.BCELoss() # поскольку у нас для каждой модели одноклассовая классификация - да или нет
        optimizer = optim.Adam(model.parameters(), lr=0.001) # у Adam хорошая сходимость 
        
        train_dataset = SeriesIntroDataset(X_train, y_train)
        val_dataset = SeriesIntroDataset(X_val, y_val)
        
        train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True) # бьём по батчам, чтоб быстро работало
        val_loader = DataLoader(val_dataset, batch_size=32)
        
        best_val_loss = float('inf')
        
        for epoch in range(epochs): # стандартный цикл обучения
            model.train()
            train_loss = 0
            for features, labels in train_loader:
                features, labels = features.to(self.device), labels.to(self.device)
                
                optimizer.zero_grad()
                outputs = model(features)
                loss = criterion(outputs.squeeze(), labels)
                loss.backward()
                optimizer.step()
                
                train_loss += loss.item()
            
            model.eval()
            val_loss = 0
            all_preds = []
            all_labels = []
            with torch.no_grad(): # теперь считаем предикты
                for features, labels in val_loader:
                    features, labels = features.to(self.device), labels.to(self.device)
                    outputs = model(features)
                    loss = criterion(outputs.squeeze(), labels)
                    val_loss += loss.item()
                    
                    preds = (outputs > 0.5).float()
                    all_preds.extend(preds.cpu().numpy())
                    all_labels.extend(labels.cpu().numpy())
            
            print(f"\n{series_name} - Epoch {epoch+1}/{epochs}")
            print(f"Train Loss: {train_loss/len(train_loader):.4f}")
            print(f"Val Loss: {val_loss/len(val_loader):.4f}")
            print(classification_report(all_labels, all_preds))
            
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                torch.save(model.state_dict(), f"best_model_{series_name}.pt") # сохраняем лучшую модель
        
        model.load_state_dict(torch.load(f"best_model_{series_name}.pt"))
        return model
    
    def train_series_models(self, series_data, epochs=10):
        for series_name, videos in tqdm(series_data.items(), desc="Обучение по сериалам"):
            X = []
            y = []
            
            for video in videos:
                features, _, labels = self.process_video(
                    video['video_path'],
                    (video['start_time'], video['end_time'])
                )
                X.append(features)
                y.append(labels)
            
            X_series = np.concatenate(X)
            y_series = np.concatenate(y) # конкатенируем, разбиваем на трейн и валидацию, обучаем
            
            X_train, X_val, y_train, y_val = train_test_split(
                X_series, y_series, test_size=0.2, random_state=42
            )
            
            model = self.train_model(X_train, y_train, X_val, y_val, series_name, epochs)
            self.models[series_name] = model
    
    def predict_intro(self, video_path, series_name, threshold=0.7):
        if series_name not in self.models:
            raise ValueError(f"Нет модели для сериала {series_name}")
            
        model = self.models[series_name]
        model.eval()
        
        features, times, _ = self.process_video(video_path)
        features_tensor = torch.FloatTensor(features).to(self.device)
        
        with torch.no_grad():
            probs = model(features_tensor).cpu().numpy().flatten()
        
        intro_segments = []
        current_segment = []
        
        for time, prob in zip(times, probs): # соединяем времена и предсказания
            if prob > threshold: # порог для определения заставки
                current_segment.append(time)
            elif current_segment:
                if len(current_segment) >= 3: # хотя бы 3 кадра пока, потом мб увеличить надо
                    intro_segments.append((
                        min(current_segment), 
                        max(current_segment)
                    ))
                current_segment = []
        
        return max(intro_segments, key=lambda x: x[1]-x[0]) if intro_segments else None
    
    def load_model(self, model_path, series_name):
        if not os.path.exists(model_path):
            raise FileNotFoundError(f"Файл модели {model_path} не найден")
    
        model = SeriesClassifier().to(self.device)
        model.load_state_dict(torch.load(model_path))
        model.eval()
    
        self.models[series_name] = model
    
    def load_models_from_directory(self, models_dir):
        if not os.path.isdir(models_dir):
            raise NotADirectoryError(f"Директория {models_dir} не существует")
    
        for model_file in os.listdir(models_dir):
            if model_file.endswith('.pt'):
                # Извлекаем название сериала из имени файла
                series_name = model_file.replace('best_model_', '').replace('.pt', '')
                model_path = os.path.join(models_dir, model_file)
                self.load_model(model_path, series_name)
    
    def evaluate_on_test(self, series_data_test):
        results = {}
        
        for series_name, videos in series_data_test.items():
            if series_name not in self.models:
                continue
                
            true_positives = 0
            total = len(videos)
            
            for video in videos:
                pred = self.predict_intro(video['video_path'], series_name)
                true_start = self.time_to_seconds(video['start_time'])
                true_end = self.time_to_seconds(video['end_time'])
                
                if pred:
                    pred_start, pred_end = pred
                    overlap = min(pred_end, true_end) - max(pred_start, true_start)
                    if overlap > 0.5 * (true_end - true_start): # метрику посчитаем по проценту перекрытия интервалов - больше или меньше 50
                        true_positives += 1
            
            accuracy = true_positives / total if total > 0 else 0 # в целом, тут у нас нет никаких вводных для метрики, поэтому accuracy
            results[series_name] = accuracy
        
        return results

In [ ]:
detector = SeriesAwareIntroDetector(frame_interval=0.5, device=device)

# обучаем на 10 эпох
detector.train_series_models(series_data_train, epochs=10)

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /tmp/xdg_cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 62.1MB/s]
Обучение по сериалам:   0%|          | 0/17 [00:00<?, ?it/s]


Баскетс - Epoch 1/10
Train Loss: 0.0176
Val Loss: 0.0093
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     11976
         1.0       0.87      0.35      0.50        37

    accuracy                           1.00     12013
   macro avg       0.93      0.68      0.75     12013
weighted avg       1.00      1.00      1.00     12013


Баскетс - Epoch 2/10
Train Loss: 0.0107
Val Loss: 0.0076
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     11976
         1.0       0.88      0.41      0.56        37

    accuracy                           1.00     12013
   macro avg       0.94      0.70      0.78     12013
weighted avg       1.00      1.00      1.00     12013


Баскетс - Epoch 3/10
Train Loss: 0.0090
Val Loss: 0.0055
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     11976
         1.0       0.89      0.65      0.75        37

    accu

Обучение по сериалам:   6%|▌         | 1/17 [19:47<5:16:42, 1187.68s/it]

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     11976
         1.0       0.82      0.76      0.79        37

    accuracy                           1.00     12013
   macro avg       0.91      0.88      0.89     12013
weighted avg       1.00      1.00      1.00     12013


Бывaeт и xyжe - Epoch 1/10
Train Loss: 0.0067
Val Loss: 0.0047
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      4799
         1.0       0.00      0.00      0.00        15

    accuracy                           1.00      4814
   macro avg       0.50      0.50      0.50      4814
weighted avg       0.99      1.00      1.00      4814



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

[NULL @ 0x55b04d037f80] Invalid NAL unit size (12559 > 4470).
[NULL @ 0x55b04d037f80] missing picture in access unit with size 4486
[h264 @ 0x55b051882640] Invalid NAL unit size (12559 > 4470).
[h264 @ 0x55b051882640] Error splitting the input into NAL units.
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x55b04f1a5ac0] stream 0, offset 0x12c01f99: partial file
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x55b04f1a5ac0] stream 1, offset 0x12c06937: partial file
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
 


3вeздный пyть: Пикap - Epoch 1/10
Train Loss: 0.1454
Val Loss: 0.0700
              precision    recall  f1-score   support

         0.0       0.97      1.00      0.99       182
         1.0       0.00      0.00      0.00         5

    accuracy                           0.97       187
   macro avg       0.49      0.50      0.49       187
weighted avg       0.95      0.97      0.96       187


3вeздный пyть: Пикap - Epoch 2/10
Train Loss: 0.0476
Val Loss: 0.0461
              precision    recall  f1-score   support

         0.0       0.98      1.00      0.99       182
         1.0       1.00      0.20      0.33         5

    accuracy                           0.98       187
   macro avg       0.99      0.60      0.66       187
weighted avg       0.98      0.98      0.97       187


3вeздный пyть: Пикap - Epoch 3/10
Train Loss: 0.0233
Val Loss: 0.0370
              precision    recall  f1-score   support

         0.0       0.99      1.00      1.00       182
         1.0       1.00 

Обучение по сериалам:  88%|████████▊ | 15/17 [1:33:11<06:03, 181.70s/it]


3вeздный пyть: Пикap - Epoch 9/10
Train Loss: 0.0057
Val Loss: 0.0215
              precision    recall  f1-score   support

         0.0       0.99      1.00      1.00       182
         1.0       1.00      0.80      0.89         5

    accuracy                           0.99       187
   macro avg       1.00      0.90      0.94       187
weighted avg       0.99      0.99      0.99       187


3вeздный пyть: Пикap - Epoch 10/10
Train Loss: 0.0025
Val Loss: 0.0224
              precision    recall  f1-score   support

         0.0       0.99      1.00      1.00       182
         1.0       1.00      0.80      0.89         5

    accuracy                           0.99       187
   macro avg       1.00      0.90      0.94       187
weighted avg       0.99      0.99      0.99       187



[NULL @ 0x55b04cf16a80] Invalid NAL unit size (3275 > 367).
[NULL @ 0x55b04cf16a80] missing picture in access unit with size 383
[h264 @ 0x55b04cf029c0] Invalid NAL unit size (3275 > 367).
[h264 @ 0x55b04cf029c0] Error splitting the input into NAL units.
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x55b0508e8340] stream 0, offset 0x12c00b5c: partial file
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x55b0508e8340] stream 0, offset 0x12c0186b: partial file
[NULL @ 0x55b04d037f80] Invalid NAL unit size (10768 > 6926).
[NULL @ 0x55b04d037f80] missing picture in access unit with size 6930
[h264 @ 0x55b04f99e8c0] Invalid NAL unit size (10768 > 6926).
[h264 @ 0x55b04f99e8c0] Error splitting the input into NAL units.
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x55b0508e8340] stream 1, offset 0x12c00f02: partial file
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x55b0508e8340] stream 1, offset 0x12c01051: partial file
[NULL @ 0x55b04d037f80] Invalid NAL unit size (19124 > 2324).
[NULL @ 0x55b04d037f80] missing picture in access unit with size 2328
[h264 @ 0x55b

Для загрузки весов модели, чтоб её не обучать:

In [27]:
detector = SeriesAwareIntroDetector()
detector.load_models_from_directory("/home/jupyter/datasphere/project")

In [ ]:
# смотрим результат
results = detector.evaluate_on_test(series_data_test)
print("\nРезультаты на тестовых данных:")
for series, accuracy in results.items():
    print(f"{series}: {accuracy:.2%}")

[NULL @ 0x55b0507e9f80] Invalid NAL unit size (9848 > 6956).
[NULL @ 0x55b0507e9f80] missing picture in access unit with size 6960
[h264 @ 0x55b0408bde40] Invalid NAL unit size (9848 > 6956).
[h264 @ 0x55b0408bde40] Error splitting the input into NAL units.
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x55b04d030b00] stream 0, offset 0x12c00b4c: partial file
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x55b04d030b00] stream 1, offset 0x12c00c50: partial file
[NULL @ 0x55b04e7c4480] Invalid NAL unit size (7767 > 4906).
[NULL @ 0x55b04e7c4480] missing picture in access unit with size 4922
[h264 @ 0x55b051579900] Invalid NAL unit size (7767 > 4906).
[h264 @ 0x55b051579900] Error splitting the input into NAL units.
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x55b05140e700] stream 0, offset 0x12c00b2d: partial file
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x55b05140e700] stream 0, offset 0x12c015ae: partial file
